In [3]:
import pandas as pd
import statsmodels.formula.api as smf

# Load datasets
df_test = pd.read_csv('test.csv')
df_weather = pd.read_csv('wetter.csv')
df_kiwo = pd.read_csv('kiwo.csv')
df_holidays = pd.read_csv('Feiertage.csv')

# Merge sales and weather data
df_test = pd.merge(df_test, df_weather, on='Datum', how='left')

# Merge with Kieler Woche data
df_test = pd.merge(df_test, df_kiwo, on='Datum', how='left')

# Define weather codes that indicate precipitation, snow, or wet conditions
rain_weather_codes = [
    # Rain related codes
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67,
    80,81,82,91,92,95,97
]

snow_weather_codes = [
    # Snow related codes
    36,37,38,39,
    68,69,70,71,72,73,74,75,76,77,78,79,
    83,84,85,86,87,88,89,90,93,94,95,96,97,99

]


# Create dry_weather column (1 for dry, 0 for wet)
df_test['rain_weather'] = df_test['Wettercode'].apply(lambda x: 1 if x in rain_weather_codes else 0)
df_test['snow_weather'] = df_test['Wettercode'].apply(lambda x: 1 if x in snow_weather_codes else 0)

# Fill missing values in KielerWoche column with 0
df_test['KielerWoche'] = df_test['KielerWoche'].fillna(0)

# Prepare holidays data
df_holidays['Feiertag'] = 1

# Merge with Feiertag data
df_final_test= pd.merge(df_test, df_holidays, on='Datum', how='left')
df_final_test['Feiertag'] = df_final_test['Feiertag'].fillna(0)

month_to_season = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
}

# Stefan Temperatur nach Jahreszeit in ranges einteilen: kalt normal warm
def analyze_seasons(df):
    # Add season column based on month
    df['Season'] = pd.to_datetime(df['Datum']).dt.month.map(month_to_season)

    # Group by season and calculate min/max temperature
    seasonal_stats = df.groupby('Season')['Temperatur'].agg(['min', 'max'])
    
    return seasonal_stats

# Analyze seasons and add season column to df_final_test_test
seasonal_stats = analyze_seasons(df_final_test)
print(seasonal_stats)

# Add season column to df_final_test_test
df_final_test['Season'] = pd.to_datetime(df_final_test['Datum']).dt.month.map(month_to_season)

# Function to categorize temperature into bins based on season
def categorize_temperature(row, seasonal_stats):
    season = row['Season']
    temp = row['Temperatur']
    min_temp = seasonal_stats.loc[season, 'min']
    max_temp = seasonal_stats.loc[season, 'max']
    bins = pd.cut([min_temp, max_temp], bins=3, retbins=True)[1]
    temp_bins = pd.cut([temp], bins=bins, labels=['low', 'middle', 'high'])
    return f"{season}_{temp_bins[0]}"

# Apply the function to create a new column in df_final_test_test
df_final_test['Season_Temp_Category'] = df_final_test.apply(lambda row: categorize_temperature(row, seasonal_stats), axis=1)

# Apply one hot encoding for Season_Temp_Category with 0 and 1
df_final_test = pd.concat([df_final_test, pd.get_dummies(df_final_test['Season_Temp_Category'], prefix='Season_Temp', drop_first=False)], axis=1)

print(df_final_test)

            min        max
Season                    
Fall     1.5000  21.412500
Spring   3.4750  19.612500
Summer  12.9000  32.671428
Winter  -4.8125  13.412500
           id       Datum  Warengruppe  Bewoelkung  Temperatur  \
0     1808011  2018-08-01            1         0.0     23.7625   
1     1808021  2018-08-02            1         0.0     26.1875   
2     1808031  2018-08-03            1         1.0     27.6625   
3     1808041  2018-08-04            1         4.0     25.1375   
4     1808051  2018-08-05            1         7.0     21.3000   
...       ...         ...          ...         ...         ...   
1825  1812226  2018-12-22            6         8.0      4.3000   
1826  1812236  2018-12-23            6         7.0      6.4500   
1827  1812246  2018-12-24            6         7.0      2.5000   
1828  1812276  2018-12-27            6         7.0      7.1250   
1829  1812286  2018-12-28            6         7.0      7.3125   

      Windgeschwindigkeit  Wettercode  Kieler

In [4]:
# Add a column to df_final_test that gives the day of the week for each date in the Datum column
df_final_test['Wochentag'] = pd.to_datetime(df_final_test['Datum']).dt.day_name()

# Create a dummy variable for weekends (1 for Saturday and Sunday, 0 otherwise)
df_final_test['is_weekend'] = df_final_test['Wochentag'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

print(df_final_test[['Datum', 'Wochentag', 'is_weekend']])

# get one hot ecoding for Warengruppe
df_final_test = pd.concat([df_final_test, pd.get_dummies(df_final_test['Warengruppe'], prefix='Warengruppe', drop_first=False)], axis=1)

df_final_test.to_csv('test_df.csv', index=False)

           Datum  Wochentag  is_weekend
0     2018-08-01  Wednesday           0
1     2018-08-02   Thursday           0
2     2018-08-03     Friday           0
3     2018-08-04   Saturday           1
4     2018-08-05     Sunday           1
...          ...        ...         ...
1825  2018-12-22   Saturday           1
1826  2018-12-23     Sunday           1
1827  2018-12-24     Monday           0
1828  2018-12-27   Thursday           0
1829  2018-12-28     Friday           0

[1830 rows x 3 columns]
